In [2]:
from model import *

In [3]:
gin.parse_config_file("config/gpt2-small.gin")
config = GPTConfig()
config

GPTConfig(block_size=1024, vocab_size=50257, n_layer=12, n_head=12, n_embd=768, batch_size=8, learning_rate=6e-05, weight_decay=0.001, eps=1e-08, betas=(0.9, 0.95), seed=42, epochs=2, training_backend='nccl', device='cuda', model_name='gpt2', clip_grad_norm_val=1.0, dtype=torch.bfloat16)

In [5]:
model = GPT.from_pretrained("gpt2")

loading weights from pretrained gpt: gpt2


In [6]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer



def generate(model,
             tokenizer,
             prompt: str,
             n_tokens_to_gen: int = 200,
             sample: bool = True,
             top_k: int = 40):
    model.eval()

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids

    for token_n in range(n_tokens_to_gen):
        with torch.no_grad():
            indices_to_input = input_ids
            next_token_logits, loss = model(indices_to_input)
            next_token_logits = next_token_logits[:, -1]

        probs = F.softmax(next_token_logits, dim=-1)
        (batch, vocab_size) = probs.shape

        if top_k is not None:
            (values, indices) = torch.topk(probs, k=top_k)
            probs[probs < values[:, -1, None]] = 0
            probs = probs / probs.sum(axis=1, keepdims=True)

        if sample:
            next_indices = torch.multinomial(probs, num_samples=1)
        else:
            next_indices = torch.argmax(probs, dim=-1)[:, None]

        input_ids = torch.cat([input_ids, next_indices], dim=1)

    output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]

    return output_completions

In [11]:
output = generate(model, tokenizer=AutoTokenizer.from_pretrained("gpt2"), prompt="User: what is the meaning of life? Assistant:", n_tokens_to_gen=100, sample=True, top_k=40)

c:\Users\sathi\miniconda3\envs\dl\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
output

'User: what is the meaning of life? Assistant: Life gives us freedom, freedom from the world. A person who lives is not to be taken as a mere living thing. Life is to be a living thing of the mind, rather than something you can just move around the world in a small room. And so on. I can never be taken as anything but that.\n\nThis is quite literally an analogy to the concept of life as an activity. Life is something which creates or changes the world with one step, an action, or the'